<a href="https://colab.research.google.com/github/avinregmi/PyTorch-Lessons/blob/master/0.Autograd_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline


Autograd: Automatic Differentiation
===================================

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our
first neural network.


The ``autograd`` package provides automatic differentiation for all operations
on Tensors. It is a define-by-run framework, which means that your backprop is
defined by how your code is run, and that every single iteration can be
different.

Let us see this in more simple terms with some examples.

Tensor
--------

``torch.Tensor`` is the central class of the package. If you set its attribute
``.requires_grad`` as ``True`` ie. ``torch.ones(2,2,requires_grad=True)``, it starts to track all operations on it. When
you finish your computation you can call ``.backward()`` and have all the
gradients computed automatically. The gradient for this tensor will be
accumulated into ``.grad`` attribute.

To stop a tensor from tracking history, you can call ``.detach()`` to detach
it from the computation history, and to prevent future computation from being
tracked.

To prevent tracking history (and using memory), you can also wrap the code block
in ``with torch.no_grad():``. This can be particularly helpful when evaluating a
model because the model may have trainable parameters with `requires_grad=True`,
but for which we don't need the gradients.

There’s one more class which is very important for autograd
implementation - a ``Function``.

``Tensor`` and ``Function`` are interconnected and build up an acyclic
graph, that encodes a complete history of computation. Each tensor has
a ``.grad_fn`` attribute that references a ``Function`` that has created
the ``Tensor`` (except for Tensors created by the user - their
``grad_fn is None``).

If you want to compute the derivatives, you can call ``.backward()`` on
a ``Tensor``. If ``Tensor`` is a scalar (i.e. it holds a one element
data), you don’t need to specify any arguments to ``backward()``,
however if it has more elements, you need to specify a ``gradient``
argument that is a tensor of matching shape.



In [0]:
import torch

Create a tensor and set requires_grad=True to track computation with it



In [0]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do an operation of tensor:



In [0]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


``y`` was created as a result of an operation, so it has a ``grad_fn``.



In [0]:
print(y.grad_fn)

Do more operations on y



In [0]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [0]:
z

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

``.requires_grad_( ... )`` changes an existing Tensor's ``requires_grad``
flag in-place. The input flag defaults to ``False`` if not given.



In [0]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


Gradients
---------

>$9x^4+2x^3+3x^2+6x+1$

>$d(9x^4+2x^3+3x^2+6x+1)/dx = 36x^3+6x^2+6x+6$ @ x = 2 = 36


### backward() and grad

y.backward() computes d(y)/dx for every parameters which has requires_grad=True and stores the gradient into grad.
<br>
x.grad += d(y)/dx

In [0]:
x = torch.tensor(2.0, requires_grad=True)
print(x)
y = 9*x**4 + 2*x**3 + 3*x**2 + 6*x +1
print(y)
print(y.grad_fn)
y.backward()
x.grad



tensor(2., requires_grad=True)
tensor(185., grad_fn=<AddBackward0>)


tensor(330.)

>$ f = 3a^2+2b^3$

>$ df/da = 6a,  df/db = 6b^2$
<br>
<br>
f.backward() will compute df/da and df/db and store into a.grad and b.grad

> $a=2 and b=3$

>$ df/da = 6a = 6(2) = 12 $

>$ df/db = 6b^2 = 6(3)^2 = 54 $

In [0]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
f = 3*a**2+2*b**3
print(f)
f.backward()
print(a.grad)
print(b.grad)


tensor(66., grad_fn=<AddBackward0>)
tensor(12.)
tensor(54.)
